# 30-Day Hospital Readmission Risk — Clinical Analytics Notebook

**Clinical aim.** Estimate 30-day readmission risk at discharge to prioritize proactive outreach (early clinic visit, medication reconciliation, home health). The model is designed to support value-based care and reduce avoidable readmissions.

**Dataset.** `data/hospital_readmissions_30k.csv`  
**Outcome.** `readmitted_30_days` ∈ {Yes, No} (mapped to 1/0)  
**Repository.** https://github.com/albertokabore/Hospital-Readmission-Prediction

**Decision framing.**  
- Primary: *Recall (Sensitivity)* — identify high-risk patients to avoid missed opportunities.  
- Secondary: *Precision* — efficient use of follow-up resources.  
- Summary: AUROC and AUPRC given class imbalance.

**Method overview.**  
- Robust preprocessing (impute, encode, scale).  
- Baselines + ensembles (DT, RF, GB, AdaBoost, Bagging, XGBoost).  
- Imbalance strategies (SMOTE, undersampling).  
- Hyperparameter tuning with `RandomizedSearchCV`.  
- Interpretability (feature importance / coefficients).


In [17]:
!pip install scikit-learn==1.2.2 seaborn==0.13.1 matplotlib==3.7.1 numpy==1.25.2 pandas==1.5.3 imbalanced-learn==0.10.1 xgboost==2.0.3 -q --user



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: C:\Users\alber\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [18]:
!pip install scikit-learn==1.2.2 seaborn==0.13.1 matplotlib==3.7.1 numpy==1.25.2 pandas==1.5.3 imbalanced-learn==0.10.1 xgboost==2.0.3 -q --user



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: C:\Users\alber\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [19]:
!pip install --upgrade -q threadpoolctl



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: C:\Users\alber\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [20]:
# Libraries to help with reading and manipulating data
import pandas as pd
import numpy as np

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To tune model, get different metric scores, and split data
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    ConfusionMatrixDisplay,
)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To impute missing values
from sklearn.impute import SimpleImputer

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To do hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# To help with model building
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# Additional essentials
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

# Display and warnings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
import warnings
warnings.filterwarnings("ignore")

RSEED = 42
sns.set(context="notebook", style="whitegrid")


ModuleNotFoundError: No module named 'sklearn'

## 1) Clinical Context & Business Value

Readmissions within 30 days are a preventable harm and a cost driver. CMS penalizes excessive readmissions; health systems seek to target high-risk discharges for early follow-up and service coordination.  
**Clinical question:** who is most likely to be readmitted in the next 30 days?


### load data

In [ ]:
from pathlib import Path

DATA_PATH = Path("data")
FILE_NAME = "hospital_readmissions_30k.csv"
TARGET = "readmitted_30_days"

csv_path = DATA_PATH / FILE_NAME
assert csv_path.exists(), f"Dataset not found: {csv_path}. Place the CSV under data/."

df = pd.read_csv(csv_path, low_memory=False)
print("Shape:", df.shape)
df.head()


## 2) Data Understanding

We profile schema, missingness, and the outcome distribution to confirm problem framing (binary classification with class imbalance is expected in clinical data).


### schema & missingness

In [ ]:
# Schema overview
buf = []
df.info(buf=buf.append)
print("\n".join(buf))

# Missingness
miss = df.isna().sum().sort_values(ascending=False)
print("\nColumns with missing values (top 20):")
display(miss[miss > 0].head(20))


### outcome normalization & class balance

In [ ]:
# Normalize outcome to {0,1}
y_raw = df[TARGET].astype(str).str.strip().str.title()
y = y_raw.map({"Yes": 1, "No": 0}).astype(int)
X = df.drop(columns=[TARGET])

print(f"Positive rate (Yes): {y.mean():.3f}")

plt.figure(figsize=(5,4))
sns.countplot(x=y_raw)
plt.title("Outcome distribution (30-day readmission)")
plt.xlabel(TARGET)
plt.ylabel("Count")
plt.tight_layout()
plt.show()


## 3) Clinical Feature Review & EDA

We separate numeric and categorical features, summarize distributions, and inspect clinically relevant categories (e.g., discharge destination) where present.


### feature typing & quick EDA

In [ ]:
cat_cols = [c for c in X.columns if X[c].dtype == "object"]
num_cols = [c for c in X.columns if c not in cat_cols]

print(f"Categorical features: {len(cat_cols)}")
print(f"Numeric features:     {len(num_cols)}")

# Numeric summary
if num_cols:
    display(X[num_cols].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T)

# Correlation (numeric only)
if len(num_cols) > 1:
    plt.figure(figsize=(8,6))
    sns.heatmap(X[num_cols].corr(numeric_only=True), cmap="coolwarm", center=0)
    plt.title("Correlation heatmap (numeric features)")
    plt.tight_layout()
    plt.show()

# Selected categorical peeks if present
for c in ["gender", "discharge_destination", "diabetes", "hypertension"]:
    if c in X.columns:
        print(f"\n{c} (top categories):")
        display(X[c].value_counts(dropna=False).head(10))


## 4) Modeling Strategy

- **Problem type:** binary classification with class imbalance.  
- **Preprocessing:** impute numeric (median); impute categorical (most frequent) + one-hot encoding; scaling (we use both StandardScaler and MinMaxScaler).  
- **Models:** DT, RF, GB, AdaBoost, Bagging, XGBoost.  
- **Imbalance handling:** SMOTE oversampling and random undersampling (inside pipeline to avoid leakage).  
- **Selection:** AUROC and AUPRC on held-out test set; recall prioritized for clinical sensitivity.


In [ ]:
# Numeric summary
display(X[num_cols].describe(percentiles=[0.01, 0.25, 0.5, 0.75, 0.99]).T)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RSEED, stratify=y
)
X_train.shape, X_test.shape


### preprocessing: Standard & MinMax pipelines

In [ ]:
numeric_standard = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

numeric_minmax = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", MinMaxScaler()),
])

categorical = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess_standard = ColumnTransformer(
    transformers=[
        ("num", numeric_standard, num_cols),
        ("cat", categorical, cat_cols),
    ]
)

preprocess_minmax = ColumnTransformer(
    transformers=[
        ("num", numeric_minmax, num_cols),
        ("cat", categorical, cat_cols),
    ]
)


### evaluation helper

In [ ]:
def evaluate_model(pipe, X_te, y_te, name="Model"):
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        proba = pipe.predict_proba(X_te)[:, 1]
    else:
        proba = pipe.decision_function(X_te)
    pred = (proba >= 0.5).astype(int)

    auc = roc_auc_score(y_te, proba)
    acc = accuracy_score(y_te, pred)
    rec = recall_score(y_te, pred)
    pre = precision_score(y_te, pred)
    f1  = f1_score(y_te, pred)

    print(f"\n=== {name} ===")
    print(f"AUROC: {auc:.3f} | ACC: {acc:.3f} | P: {pre:.3f} | R: {rec:.3f} | F1: {f1:.3f}")
    print("\nClassification report:")
    print(classification_report(y_te, pred, digits=3))

    ConfusionMatrixDisplay(confusion_matrix(y_te, pred)).plot(cmap="Blues")
    plt.title(f"{name} — Confusion Matrix")
    plt.show()

    RocCurveDisplay.from_predictions(y_te, proba)
    plt.title(f"{name} — ROC Curve")
    plt.show()

    PrecisionRecallDisplay.from_predictions(y_te, proba)
    plt.title(f"{name} — Precision-Recall Curve")
    plt.show()

    return {"AUROC": auc, "ACC": acc, "P": pre, "R": rec, "F1": f1}


### baselines on StandardScaler

In [ ]:
models = {
    "DecisionTree": DecisionTreeClassifier(random_state=RSEED),
    "RandomForest": RandomForestClassifier(n_estimators=300, random_state=RSEED, n_jobs=-1),
    "GradientBoosting": GradientBoostingClassifier(random_state=RSEED),
    "AdaBoost": AdaBoostClassifier(random_state=RSEED),
    "Bagging_DT": BaggingClassifier(
        base_estimator=DecisionTreeClassifier(random_state=RSEED),
        n_estimators=50, random_state=RSEED, n_jobs=-1
    ),
    "XGBoost": XGBClassifier(
        random_state=RSEED, n_estimators=300, learning_rate=0.1,
        subsample=0.9, colsample_bytree=0.9, max_depth=6,
        eval_metric="logloss", n_jobs=-1
    ),
}

results_std = {}
for name, clf in models.items():
    pipe = Pipeline([("prep", preprocess_standard), ("clf", clf)])
    pipe.fit(X_train, y_train)
    metrics = evaluate_model(pipe, X_test, y_test, name=f"{name} (std)")
    results_std[name] = metrics

pd.DataFrame(results_std).T.sort_values("AUROC", ascending=False)


### baselines on MinMaxScaler to use MinMax explicitly

In [ ]:
results_mm = {}
for name, clf in models.items():
    pipe = Pipeline([("prep", preprocess_minmax), ("clf", clf)])
    pipe.fit(X_train, y_train)
    metrics = evaluate_model(pipe, X_test, y_test, name=f"{name} (minmax)")
    results_mm[name] = metrics

pd.DataFrame(results_mm).T.sort_values("AUROC", ascending=False)


### imbalance handling: SMOTE & undersampling

In [ ]:
# Build resampling pipelines for strong models (RF and XGB)
rf_smote = ImbPipeline(steps=[
    ("prep", preprocess_standard),
    ("smote", SMOTE(random_state=RSEED)),
    ("clf", RandomForestClassifier(n_estimators=400, random_state=RSEED, n_jobs=-1)),
])

rf_under = ImbPipeline(steps=[
    ("prep", preprocess_standard),
    ("under", RandomUnderSampler(random_state=RSEED)),
    ("clf", RandomForestClassifier(n_estimators=400, random_state=RSEED, n_jobs=-1)),
])

xgb_smote = ImbPipeline(steps=[
    ("prep", preprocess_standard),
    ("smote", SMOTE(random_state=RSEED)),
    ("clf", XGBClassifier(
        objective="binary:logistic", eval_metric="logloss",
        random_state=RSEED, n_estimators=300, learning_rate=0.1,
        subsample=0.9, colsample_bytree=0.9, max_depth=6, n_jobs=-1
    )),
])

for label, pipe in {
    "RF + SMOTE": rf_smote,
    "RF + UnderSample": rf_under,
    "XGB + SMOTE": xgb_smote,
}.items():
    pipe.fit(X_train, y_train)
    _ = evaluate_model(pipe, X_test, y_test, name=label)


### hyperparameter tuning with RandomizedSearchCV for XGB + SMOTE

In [ ]:
xgb_base = ImbPipeline(steps=[
    ("prep", preprocess_standard),
    ("smote", SMOTE(random_state=RSEED)),
    ("clf", XGBClassifier(
        objective="binary:logistic", eval_metric="logloss",
        random_state=RSEED, n_jobs=-1
    )),
])

param_dist = {
    "clf__n_estimators": [200, 300, 400, 600, 800],
    "clf__max_depth": [3, 4, 5, 6, 8],
    "clf__learning_rate": [0.03, 0.05, 0.1, 0.2],
    "clf__subsample": [0.7, 0.8, 0.9, 1.0],
    "clf__colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "clf__min_child_weight": [1, 3, 5, 7],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RSEED)

search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    n_jobs=-1,
    cv=cv,
    random_state=RSEED,
    verbose=1,
)

search.fit(X_train, y_train)
print("Best CV AUROC:", search.best_score_)
print("Best params:", search.best_params_)

xgb_tuned = search.best_estimator_
_ = evaluate_model(xgb_tuned, X_test, y_test, name="XGB + SMOTE (tuned)")


### feature importance / explainability

In [ ]:
def get_feature_names(preprocessor, cat_cols, num_cols):
    names = []
    names.extend(num_cols)
    ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]
    names.extend(list(ohe.get_feature_names_out(cat_cols)))
    return names

def show_importances(pipeline, title):
    prep = pipeline.named_steps["prep"]
    names = get_feature_names(prep, cat_cols, num_cols)
    clf = pipeline.named_steps["clf"]
    if hasattr(clf, "feature_importances_"):
        imp = pd.Series(clf.feature_importances_, index=names).sort_values(ascending=False)
        top20 = imp.head(20)
        display(top20)
        plt.figure(figsize=(8,6))
        sns.barplot(x=top20.values, y=top20.index, orient="h")
        plt.title(title)
        plt.tight_layout()
        plt.show()
    else:
        print("Model has no feature_importances_ attribute.")

# Show on tuned XGB (if tuned), else on RF+SMOTE
try:
    show_importances(xgb_tuned, "Top Predictors — XGB + SMOTE (tuned)")
except NameError:
    show_importances(rf_smote, "Top Predictors — RF + SMOTE")


## 5) Clinical Interpretation & Decision Support

- **Sensitivity first:** missing a high-risk patient is a lost prevention opportunity. Tune threshold for desired recall (e.g., 0.75) and monitor precision to manage workload.  
- **Actionability:** high-risk flag at discharge → care coordination outreach in 48–72 hours; medication reconciliation; early clinic visit; home health referral when appropriate.  
- **Governance:** establish clinical oversight, periodic performance review, fairness checks across subgroups (age, sex, insurance), and calibration monitoring.


### threshold tuning utility

In [ ]:
def threshold_sweep(y_true, scores, grid=np.linspace(0.1, 0.9, 9)):
    rows = []
    for t in grid:
        pred = (scores >= t).astype(int)
        rows.append({
            "threshold": t,
            "precision": precision_score(y_true, pred),
            "recall": recall_score(y_true, pred),
            "f1": f1_score(y_true, pred),
        })
    return pd.DataFrame(rows)

# Example with tuned XGB scores if available (falls back gracefully)
try:
    if hasattr(xgb_tuned.named_steps["clf"], "predict_proba"):
        s = xgb_tuned.predict_proba(X_test)[:, 1]
    else:
        s = xgb_tuned.decision_function(X_test)
except NameError:
    if hasattr(rf_smote.named_steps["clf"], "predict_proba"):
        s = rf_smote.predict_proba(X_test)[:, 1]
    else:
        s = rf_smote.decision_function(X_test)

sweep = threshold_sweep(y_test, s)
display(sweep)

plt.figure(figsize=(6,4))
sns.lineplot(data=sweep, x="threshold", y="recall", marker="o", label="Recall")
sns.lineplot(data=sweep, x="threshold", y="precision", marker="o", label="Precision")
sns.lineplot(data=sweep, x="threshold", y="f1", marker="o", label="F1")
plt.title("Threshold tuning (precision–recall trade-off)")
plt.tight_layout()
plt.show()


## 6) Save Artifacts & Next Steps

**Artifacts.** Save fully-fitted pipelines (preprocessing included) for downstream scoring.  
**Next steps.** Calibration, subgroup analysis, prospective validation, and operational integration in the discharge workflow.


### save models

In [ ]:
import os, joblib
os.makedirs("artifacts", exist_ok=True)

saved = {}
# Save best candidates if they exist; ignore if not fitted in this session
for name, pipe in [
    ("rf_smote", 'rf_smote'),
    ("rf_under", 'rf_under'),
    ("xgb_smote", 'xgb_smote'),
    ("xgb_tuned", 'xgb_tuned'),
]:
    try:
        obj = globals()[pipe]
        joblib.dump(obj, f"artifacts/{name}.joblib")
        saved[name] = "saved"
    except Exception as e:
        saved[name] = f"skip ({e.__class__.__name__})"

saved


## Train and test partition

We use a stratified split to preserve the outcome rate.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape


## Modeling strategy

Rationale  
Logistic Regression offers transparency for clinical stakeholders and supports coefficient based interpretation.  
Random Forest and Gradient Boosting often improve predictive performance on tabular data.

Preprocessing  
Numeric imputation uses median.  
Categorical imputation uses most frequent value followed by one hot encoding.  


In [ ]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), num_cols),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    n_jobs=None
)

models = {
    "LogReg_balanced": LogisticRegression(max_iter=1000, class_weight="balanced"),
    "RandomForest": RandomForestClassifier(
        n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced"
    ),
    "GradientBoosting": GradientBoostingClassifier(random_state=42)
}

pipelines = {
    name: Pipeline(steps=[("preprocess", preprocess), ("clf", clf)])
    for name, clf in models.items()
}
list(pipelines.keys())


## Model fitting and core evaluation

We report AUROC and AUPRC.  
We print a full classification report and a confusion matrix.  


In [ ]:
results = {}
reports = {}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)

    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        proba = pipe.predict_proba(X_test)[:, 1]
    else:
        proba = pipe.decision_function(X_test)

    pred = (proba >= 0.5).astype(int)

    au
